In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from scipy.io import wavfile
import os.path
import IPython.display
import seaborn as sns
import librosa
import librosa.display
import soundfile

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import confusion_matrix

import tensorflow as tf
from tensorflow.keras import utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Conv1D, MaxPooling1D, Flatten, BatchNormalization
from keras import optimizers

import warnings

In [2]:
!pip install pydub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import re
import os
import wave
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pydub import AudioSegment
from IPython.display import Audio, display
import librosa as lib
import librosa.display

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPool2D
from tensorflow.keras import Model

from sklearn.model_selection import train_test_split

TensorFlow version: 2.12.0


In [5]:
PATH = "/content/drive/MyDrive/Colab Notebooks/Speech_Emotion/Crema"
AUDIO_PATH = "/content/drive/MyDrive/Colab Notebooks/Speech_Emotion/Crema/1001_DFA_ANG_XX.wav"

In [6]:
def zeroCrossingRate(audio):
  return lib.feature.zero_crossing_rate(audio).mean()

In [7]:
def energy(audio):
  # using a spectrogram will give a more accurate representation
  # of energy over time because its frames can be windowed
  S, phase = lib.magphase(lib.stft(audio))
  return lib.feature.rms(S=S).mean()

In [8]:
def melSpectrogram(audio, sr):
  mel_spectrogram = lib.feature.melspectrogram(y=audio, sr=sr, n_fft=200, n_mels=64)
  log_mel_spectrogram = librosa.power_to_db(mel_spectrogram)
  return log_mel_spectrogram

In [9]:
def chromaStft(audio, sr):
  stft = np.abs(librosa.stft(audio))
  return librosa.feature.chroma_stft(S=stft, sr=sr, n_fft=200).mean()

In [10]:
def mfcc(data, sr):
  return librosa.feature.mfcc(y=data, sr=sr, n_fft=200).mean()

In [11]:
def tonnetz(data, sr):
  return librosa.feature.tonnetz(y=data, sr=sr).mean();

In [12]:
def playAudio(audio_file):
  audio = AudioSegment.from_wav(audio_file)
  # Play the audio
  audio.export('temp_audio.wav', format='wav')
  audio_data = open('temp_audio.wav', 'rb').read()
  display(Audio(audio_data))
  # Delete the temporary audio file
  os.remove('temp_audio.wav')

In [13]:
def visualize_waveform(audio, sr):
    plt.figure(figsize=(12, 4))
    plt.plot(audio)
    plt.xlabel('Time')
    plt.ylabel('Amplitude')
    plt.title('Waveform')
    plt.show()

In [14]:
def splitData(dataset_path):
  audio_files = glob.glob(os.path.join(dataset_path, "*.wav"))
  audio_train, audio_test = train_test_split(audio_files, test_size=0.3, random_state=69, shuffle=True)
  return audio_train, audio_test

In [15]:
D_train, D_test = splitData(PATH)

In [16]:
print(len(D_train), len(D_test))

5209 2233


In [17]:
D_test[0]

'/content/drive/MyDrive/Colab Notebooks/Speech_Emotion/Crema/1066_DFA_HAP_XX.wav'

In [18]:
def getMaxLen(audio_files):
  max_length = 0
  for audio_file in audio_files:
    audio, _ = librosa.load(audio_file)
    length = len(audio)
    max_length = max(max_length, length)
  return max_length

In [19]:
def loadAudio(audio_files):
  classes = ["SAD", "ANG", "DIS", "FEA", "HAP", "NEU"]
  # max_length = getMaxLen(audio_files)
  max_length = 200
  D, Y = [], []
  for audio_file in audio_files: 
    # load the audio file
    audio, sr = lib.load(audio_file, sr=4444)
    # # extract zero crossing rate
    # zcr = zeroCrossingRate(audio)
    # # extract energy
    # rms = energy(audio)
    # extract mel spectrogram
    mel_spec = melSpectrogram(audio, sr)
    # # chroma stft
    # cs = chromaStft(audio, sr)
    # mfc = mfcc(audio, sr)
    # ton = tonnetz(audio, sr)
    # combined_features = np.concatenate((np.pad(audio, (0, max_length - len(audio))), [zcr, rms, mel_spec, cs, mfc]))
    # combined_features = [zcr, rms, mel_spec, cs, mfc]
    # print(len(combined_features))
    D.append(mel_spec)
    for cls in classes:
      if re.search(cls, audio_file): Y.append(cls)
  return D, Y

In [60]:
D, Y = loadAudio(D_train)

In [61]:
unicheck = np.unique(Y, axis=0)
unicheck

array(['ANG', 'DIS', 'FEA', 'HAP', 'NEU', 'SAD'], dtype='<U3')

In [62]:
df = pd.DataFrame(Y)
df.head()

,0
0,FEA
1,FEA
2,FEA
3,SAD
4,HAP


In [63]:
# Use the get_dummies() method to one-hot encode the labels
one_hot_labels = pd.get_dummies(df)

# Convert the DataFrame to a NumPy array
one_hot_labels = one_hot_labels.to_numpy()

print(one_hot_labels.shape)

(5209, 6)


In [64]:
Y = one_hot_labels
Y.shape

(5209, 6)

In [65]:
for i, array in enumerate(D):
    D[i] = np.pad(array, ((0, 0), (0, 256 - array.shape[1])))
D = np.array(D)

In [66]:
D.shape

(5209, 64, 256)

In [67]:
x_train, x_test, y_train, y_test = train_test_split(np.array(D), np.array(Y), test_size=0.1)
print((x_train.shape, y_train.shape, x_test.shape, y_test.shape))

((4688, 64, 256), (4688, 6), (521, 64, 256), (521, 6))


In [68]:
x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], x_train.shape[2], 1))
x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], x_test.shape[2], 1))

In [69]:
print(x_train.shape, x_test.shape)

(4688, 64, 256, 1) (521, 64, 256, 1)


In [92]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

input_shape = (64, 256, 1)

model = Sequential()

# Layer 1: Convolutional Layer
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(x_train.shape[1], x_train.shape[2], 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Layer 2: Convolutional Layer
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Layer 3: Convolutional Layer
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Layer 4: Flatten Layer
model.add(Flatten())

# Layer 5: Fully Connected Layer
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))

# Layer 6: Fully Connected Layer
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

# Layer 7: Output Layer
model.add(Dense(6, activation='sigmoid'))

model.compile(loss='categorical_crossentropy',
             optimizer=optimizers.Adam(lr=0.0005),
             metrics=['accuracy'])

model.summary()

Model: "sequential_24"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_100 (Conv2D)         (None, 62, 254, 32)       320       
                                                                 
 max_pooling2d_81 (MaxPoolin  (None, 31, 127, 32)      0         
 g2D)                                                            
                                                                 
 conv2d_101 (Conv2D)         (None, 29, 125, 64)       18496     
                                                                 
 max_pooling2d_82 (MaxPoolin  (None, 14, 62, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_102 (Conv2D)         (None, 12, 60, 128)       73856     
                                                                 
 max_pooling2d_83 (MaxPoolin  (None, 6, 30, 128)     

In [93]:
history = model.fit(x_train, y_train,
                    epochs=50,
                    batch_size=128,
                    validation_data=(x_test, y_test))

Epoch 1/50
37/37 [==============================] - 3s 61ms/step - loss: 2.0012 - accuracy: 0.1924 - val_loss: 1.7093 - val_accuracy: 0.2745
Epoch 2/50
37/37 [==============================] - 2s 52ms/step - loss: 1.6667 - accuracy: 0.2901 - val_loss: 1.5358 - val_accuracy: 0.3877
Epoch 3/50
37/37 [==============================] - 2s 54ms/step - loss: 1.5959 - accuracy: 0.3300 - val_loss: 1.5354 - val_accuracy: 0.3954
Epoch 4/50
37/37 [==============================] - 2s 54ms/step - loss: 1.5505 - accuracy: 0.3558 - val_loss: 1.4692 - val_accuracy: 0.4050
Epoch 5/50
37/37 [==============================] - 2s 52ms/step - loss: 1.5110 - accuracy: 0.3744 - val_loss: 1.4576 - val_accuracy: 0.4069
Epoch 6/50
37/37 [==============================] - 2s 56ms/step - loss: 1.4889 - accuracy: 0.3763 - val_loss: 1.4346 - val_accuracy: 0.4203
Epoch 7/50
37/37 [==============================] - 2s 56ms/step - loss: 1.4740 - accuracy: 0.3925 - val_loss: 1.4123 - val_accuracy: 0.4472
Epoch 8/50
37